
-<h1><center>  构建情感词典 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取</center></h1>


 


## 目标: 
-	开发一个管道来创建一个带对比度分析的自定义情感词典
  - 促进情绪建模的数据标签
  - 定义一种编程方法，用于在用户选择的语料库中创建情感词典

**SumUp对比分析的工作原理是用语库中两种不同类别的文档，由用户根据元数据或内容定义**

## 数据:
-	任何文档集合，其中至少有一部分文档标记为情感类别，如 POSITIVE(正面) / NEUTRAL(中性) / NEGATIVE(负面)


## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建

-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	对比主题建模 API
 - 	*api_instance.post_topic_contrast_api(payload)*
 

-	文件分类 API
 - 	*api_instance.post_doc_classify_api(payload)*

## 方法:

### 1.	培训，验证，测试数据集准备
-	创建包含所有相关文档的Nucleus数据集


-   必须标记输入文档
    - 这些文档都存储在CSV或JSON中，并且一列数据对应于情感标签
    - 或者，您需要在Nucleus中构建数据集时将情绪标签指定为额外的元数据字段
    

In [ ]:
print('---- Train / Validate / Test dataset ----')
folder = 'Sellside_research'         
dataset = 'Sellside_research'# str | 文件的目标数据集。

# 以递归方式从文件夹构建文件
# 其中每个采用以下格式：
# {'filename': filename,   # 要上传的文件名。 必填
#  'metadata': {           # 该文件的元数据。 选填
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }

# 如果文档不是CSV或JSON，则必须在file_iter对象中指定情感标签作为额外的元数据字段。

# 如果您正在读取已提供情绪标签的文件，则无需传递file_dict中的“元数据”（metadata)

file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        file_dict = {'filename': os.path.join(root, file),
                     'metadata': {'sentiment': 'positive' # 这里构建一些逻辑来决定如何分配 POS / NEU / NEG
                                }}
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

### 2.	未标记的数据集准备
-	创建包含所有相关文档的Nucleus数据集


-   输入文档未标记

In [ ]:
print('---- Dataset to label ----')
folder = 'Sellside_research_unlabeled'         
dataset = 'Sellside_research_unlabeled'# 字符串 | 将加入文档的目标数据集。

# 以递归方式从文件夹构建文件。
# 其中每个文件采用以下格式：
# {'filename': filename,   # 要上传的文件名。 必填  }

file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        file_dict = {'filename': os.path.join(root, file)}
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

### 3. 加速数据标签化

-     在此例子中，我们将一类文档定义为具有正面情绪。 第二类有负面情绪
-     我们在训练集上提取对比主题，用TopicContrastModel分隔这两个类别
-     我们根据创建的日期将数据集划分为训练，验证和测试集

In [ ]:
metadata_selection_contrast = {"sentiment": ["positive", "negative"]} # dict | 元数据选择定义了两类文档，以便相互对比和总结

query = '' # str | 数据集语言特定的全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 是否考虑每类文档的语法以帮助对比分析（可选）（默认为False）
num_keywords = 20 # integer | 从数据集中提取的比较主题的关键字数。 (可选) (默认为50)
remove_redundancies = False # bool | 如果为True，则此选项会从分析中删除准重复项。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）

payload = nucleus_api.TopicContrastModel(dataset='Sellside_research', 
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        period_start='2017-01-01',
                                        period_end='2018-01-01')
api_response = api_instance.post_topic_contrast_api(payload)

print('Contrasted Topic')
print('    Keywords:', api_response.result.keywords)
print('    Keywords Weight:', api_response.result.keywords_weight)

-     确定此对比主题在验证数据集上对语料库的情感标记, 执行的效果
-     我们进一步详细说明如何微调情绪贴标机

In [ ]:
fixed_topics = {"keywords": api_response.result.keywords, "weights": api_response.result.keywords_weight} # dict | 对比主题用于分隔两类文件。 重量可选
metadata_selection_contrast = {"sentiment": ["positive", "negative"]} # dict | 元数据选择定义了文档可以分类的两类文档

query = '' # str | 数据集语言特定的全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选的）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 如果为True，则分类器将在内容变量之上包含与语法相关的变量（可选）（默认为False）
threshold = 0 # float | Threshold value for a document exposure to the contrastic topic, above which the document is assigned to class 1 specified through metadata_selection. （可选）（默认为0）
remove_redundancies = False # bool | 如果为True，则此选项会从分析中删除准重复项。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）


payload = nucleus_api.DocClassifyModel(dataset="Sellside_research",
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        validation_phase=True, # 此参数告诉API数据已标记，因此生成perf度量标准
                                        period_start='2018-01-01',
                                        period_end='2019-01-01')
api_response = api_instance.post_doc_classify_api(payload)

print('Detailed Results')
print('    Docids:', api_response.result.detailed_results.docids)
print('    Exposure:', api_response.result.detailed_results.exposures)
print('    Estimated Category:', api_response.result.detailed_results.estimated_class)
print('    Actual Category:', api_response.result.detailed_results.true_class)
print('\n')

print('Perf Metrics')
print('    Accuracy:', api_response.result.perf_metrics.hit_rate)
print('    Recall:', api_response.result.perf_metrics.recall)
print('    Precision:', api_response.result.perf_metrics.precision)

-     对标签模型满意后，可以将其应用于测试数据

In [ ]:
fixed_topics = {"keywords": api_response.result.keywords, "weights": api_response.result.keywords_weight} # dict | 对比主题用于分隔两类文件。 重量可选
metadata_selection_contrast = {"sentiment": ["positive", "negative"]} # dict | 元数据选择定义了文档可以分类的两类文档

query = '' # str | Dataset-language-specific全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选的）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 如果为True，则分类器将在内容变量之上包含与语法相关的变量（可选）（默认为False）
threshold = 0 # float | Threshold value for a document exposure to the contrastic topic, above which the document is assigned to class 1 specified through metadata_selection. （可选）（默认为0）
remove_redundancies = False # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default False)


payload = nucleus_api.DocClassifyModel(dataset="Sellside_research",
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        validation_phase=True, # 此参数告诉API数据已标记，因此生成perf度量标准
                                        period_start='2019-01-01',
                                        period_end='2019-07-01')
api_response = api_instance.post_doc_classify_api(payload)

print('Detailed Results')
print('    Docids:', api_response.result.detailed_results.docids)
print('    Exposure:', api_response.result.detailed_results.exposures)
print('    Estimated Category:', api_response.result.detailed_results.estimated_class)
print('    Actual Category:', api_response.result.detailed_results.true_class)
print('\n')

print('Perf Metrics')
print('    Accuracy:', api_response.result.perf_metrics.hit_rate)
print('    Recall:', api_response.result.perf_metrics.recall)
print('    Precision:', api_response.result.perf_metrics.precision)

-     您现在可以在未标记的数据集上运行上述模型
-     您将为每个文档检索“estimated_class”，从而完成数据集标注
-     您可以对任何一对情绪标签重复此过程，交叉验证未标记数据集的情绪标签

In [ ]:
fixed_topics = {"keywords": api_response.result.keywords, "weights": api_response.result.keywords_weight} # dict | 对比主题用于分隔两类文件。 重量可选
metadata_selection_contrast = {"sentiment": ["positive", "negative"]} # dict | 元数据选择定义了文档可以分类的两类文档

query = '' # str | Dataset-language-specific全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 如果为True，则分类器将在内容变量之上包含与语法相关的变量（可选）（默认为False）
threshold = 0 # float | Threshold value for a document exposure to the contrastic topic, above which the document is assigned to class 1 specified through metadata_selection. （可选）（默认为0）
remove_redundancies = False # bool | 如果为True，则此选项会从分析中删除准重复项。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）


payload = nucleus_api.DocClassifyModel(dataset="Sellside_research_unlabeled",
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        validation_phase=False)
api_response = api_instance.post_doc_classify_api(payload)

print('Detailed Results')
print('    Docids:', api_response.result.detailed_results.docids)
print('    Estimated Category:', api_response.result.detailed_results.estimated_class)

### 4. 生成情感词典

-     使用整个数据集：训练/验证/测试数据+上一步中标记的数据
-     生成最能与上述任何两个标签形成对比的主题
-     您可以对任何一对情绪标签重复此过程，交叉验证每个单词的情绪标签

In [ ]:
print('---- Complete dataset ----')
folder = 'Sellside_research_combined'         
dataset = 'Sellside_research_combined'# str | 将插入文件的目标数据集。

# 以递归方式从文件夹构建文件。
# 每个文件采用以下格式：
# {'filename': filename,   # 要上传的文件名。 必填
#  'metadata': {           # 该文件的元数据。 可选
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }

# 如果文档不是CSV或JSON，则必须在file_iter对象中指定情感标签作为额外的元数据字段。

# 如果您正在读取已提供情绪标签的文件，则无需传递file_dict中的“元数据”

file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        file_dict = {'filename': os.path.join(root, file),
                     'metadata': {'sentiment': 'positive' # 这里传递上一步中获得的标签
                                }}
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

In [ ]:
metadata_selection_contrast = {"sentiment": ["positive", "negative"]} # dict | 元数据选择定义了两类文档，以便相互对比和总结

query = '' # str | Dataset-language-specific全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 指定是否考虑每类文档的语法方面以帮助对比它们（可选）（默认为False）
num_keywords = 20 # integer | 从数据集中提取的比较主题的关键字数。 (可选) (默认为50)
remove_redundancies = False # bool | 如果为True，则此选项会从分析中删除准重复项。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。（可选）（默认为False）

payload = nucleus_api.TopicContrastModel(dataset='Sellside_research_combined', 
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words)
api_response = api_instance.post_topic_contrast_api(payload)

print('Contrasted Topic')
print('    Keywords:', api_response.result.keywords)
print('    Keywords Weight:', api_response.result.keywords_weight)

### 5.	微调

#### a.	从对比度分析中排除某些内容

-   通过使用对比度分析API中的`custom_stop_words`参数，排除无关的关键字/主题以定制对比度分析


-	在语料库中的文档中提取对比主题的关键字，并打印这些主题的关键字

In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Sellside_research',                         
                            query='',                       
                            num_topics=8, 
                            num_keywords=8,
                            metadata_selection=metadata_selection_contrast)
api_response = api_instance.post_topic_api(payload)        
    
for i, res in enumerate(api_response.result.topics):
    print('Topic', i, ' keywords: ', res.keywords)    
    print('---------------')

使用您的专业知识，可以确定特定主题或关键字是否没有足够的差异来促进对比度分析。

然后，您可以通过创建包含这些单词的`custom_stop_words`变量来定制对比度分析。 如下所示，初始化变量并将其传递到第2节主代码的有效负载（payload)中：

In [1]:
custom_stop_words = ["disclaimer", "disclosure"] # str | 停用词列表。 （可选的）

#### b. 为您的对比主题指定`metadata_selection_contrast`

-     对比两个不同实体的文件

In [ ]:
metadata_selection_contrast = {"research_analyst": ["MS", "JPM"]}

-     对比包含不同关键字的文档

In [ ]:
metadata_selection_contrast = {"content": "fundamentals"}

#### c. 微调对比主题
**num_keywords**: “num_keywords”越大，在对比主题中保留的单词就越多，而附加的单词对分离你所处理的两类情感的影响就越小

**syntax_variables**: 如果为True，则某些词性功能会自动包含在对比主题模型中。 如果某些作者的写作风格差异很大，这可能会有所帮助。 社交媒体数据和新闻频繁发生这种情况。 它不太可能出现在机构出版物中

**threshold**: 这是文档必须具有的最小曝光，以分配给您定义的category_1的对比主题。 完美模型的阈值为0（默认值）。 您可能会发现，通过选择不同的值，可以在验证中获得更高的性能指标。 这可以在用于训练和验证的较小样本中特别解释，或者如果在对比主题中存在作为关键词出现的通用词

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 